In [20]:
import pandas as pd
import time
import requests
import msal
from io import StringIO
from powerbiclient import Report, models
from powerbiclient.authentication import DeviceCodeLoginAuthentication

In [ ]:
# Variables

AUTHORITY_URL = f'https://login.microsoftonline.com/organizations'
CLIENT_ID = '04bb970d-3099-4845-b81d-92e23362f261'
SCOPE = ["https://api.fabric.microsoft.com/Dataset.ReadWrite.All",
    "https://api.fabric.microsoft.com/Dashboard.ReadWrite.All",
    "https://api.fabric.microsoft.com/Item.ReadWrite.All",
    "https://api.fabric.microsoft.com/Workspace.ReadWrite.All",
    "https://api.fabric.microsoft.com/Report.ReadWrite.All",
    "https://api.fabric.microsoft.com/Content.Create"]
GROUP_ID = 'ceb0d1d0-6226-4aef-b245-fac7f89bb52e'
POWER_BI_API_URL = f'https://api.powerbi.com/v1.0/myorg/'
TENANT_ID = '057866cb-0e0f-4818-bd4a-0255845df359'

In [ ]:
# Authentication

# Variables
access_token = ''

# Authenticate and get access token
if not access_token:
    app = msal.PublicClientApplication(
        CLIENT_ID,
        authority=AUTHORITY_URL
    )
    result = app.acquire_token_interactive(scopes=SCOPE)
    if 'access_token' in result:
        g = result['access_token']
        print('Access token obtained successfully by interactive login')
    else:
        print(f'Error obtaining access token: {result}, trying username and password')
        app = msal.PublicClientApplication(CLIENT_ID, authority=TENANT_ID)
        username = input('Enter username: ')
        password = input('Enter password: ')
        result = app.acquire_token_by_username_password(username, password, SCOPE)
        if 'access_token' in result:
            access_token = result['access_token']
            print('Access token obtained successfully by username and password')
        else:
            print(f'Error obtaining access token: {result}')

# Headers
headers = {'Authorization': f'Bearer {access_token}'}


In [21]:
auth = DeviceCodeLoginAuthentication()
#parameters
report_id = '025bdeaf-cf4e-40b9-a4c6-01446cbdff3f'
report = Report(group_id='ceb0d1d0-6226-4aef-b245-fac7f89bb52e', report_id=report_id,  auth=auth)

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DCXPLTW55 to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


In [22]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

In [23]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

In [24]:
# Render report
report

Report()

In [25]:
time.sleep(15)

In [26]:
#GET all pages
pages = report.get_pages()
print(pages)

[{'name': 'ad7629625350de3c2f92', 'displayName': 'Customers', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}, {'name': '2d7047f129c3f2484ff2', 'displayName': 'Main Offices', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}, {'name': '38741aa4f17a9ebc9995', 'displayName': 'Product_Size_Weight_Color', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}, {'name': '50c1abe0586dfc224c03', 'displayName': 'Sales_quantity_amount_monthCategory', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallp

In [27]:
# Define the types of visuals to export data for
visual_types_to_export = ['barChart', 'lineChart', 'pieChart', 'clusteredColumnChart', 'table', 'tableEx', 'map', 'slicer', 'lineClusteredColumnComboChart', 'shapeMap', 'decompositionTreeVisual', 'card']

# Iterate over the visuals and export data for the specified types
for page in pages:
    page_name = page['name']
    report.set_active_page(page_name)
    page_display_name = page['displayName']
    visuals = report.visuals_on_page(page_name)
    for visual in visuals:
        if visual['type'] in visual_types_to_export:
            try:
                summarized_exported_data = report.export_visual_data(page_name, visual['name'], rows=20)
                data = StringIO(summarized_exported_data)
                # Load data into pandas DataFrame
                df = pd.read_csv(data, sep=",")
                # Store the DataFrame as a CSV file in the 'csv' folder
                csv_file_path = f'csv/{visual["type"]}_{visual["title"].replace(" ", "_")}_{visual['name']}_{page_display_name.replace(" ", "_")}.csv'
                df.to_csv(csv_file_path, index=False)
                print(f"Data for visual '{visual["title"].replace(" ", "")}' of type '{visual['type']}' exported successfully.")
            except Exception as e:
                print(f"Could not export data for visual '{visual["title"].replace(" ", "")}' of type '{visual['type']}': {e}")
    page['isActive'] = False